In [ ]:
import pandas as pd

def adjust_and_aggregate(df):
    # Step 1: Count how many months each (ISIN, Fund House) combination appears
    counts = df.groupby(['ISIN', 'Fund House']).size().reset_index(name='Month_Count')
    
    # Step 2: Merge the counts back with the original dataframe
    df = df.merge(counts, on=['ISIN', 'Fund House'], how='left')
    
    # Step 3: Adjust USD Difference and % Difference for ISINs that appear in all 3 months
    df['Adjusted_USD_Difference'] = df.apply(lambda row: row['USD Difference'] / 3 if row['Month_Count'] == 3 else row['USD Difference'], axis=1)
    df['Adjusted_%_Difference'] = df.apply(lambda row: row['% Difference'] / 3 if row['Month_Count'] == 3 else row['% Difference'], axis=1)
    
    # Step 4: Aggregate by Fund House to get the total adjusted USD difference
    result = df.groupby('Fund House').agg({
        'Adjusted_USD_Difference': 'sum'
    }).reset_index()
    
    # Step 5: Find the Fund House with the highest total USD difference
    top_fund_house = result.loc[result['Adjusted_USD_Difference'].idxmax()]
    
    return result, top_fund_house

# Example usage
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Jan', 'Feb', 'Mar', 'Jan', 'Feb'],
    'ISIN': ['123', '123', '123', '456', '456', '456', '789', '789'],
    'Trader Name': ['Trader A', 'Trader A', 'Trader A', 'Trader B', 'Trader B', 'Trader B', 'Trader C', 'Trader C'],
    'USD Difference': [1000, 2000, -500, 1500, 2500, 2000, -300, -200],
    '% Difference': [10, 20, -5, 15, 25, 20, -3, -2],
    'Fund House': ['Fund X', 'Fund X', 'Fund X', 'Fund Y', 'Fund Y', 'Fund Y', 'Fund Z', 'Fund Z']
}

df = pd.DataFrame(data)

# Get the results
aggregated_data, top_fund_house = adjust_and_aggregate(df)

print("Aggregated Data:")
print(aggregated_data)

print("\nTop Fund House:")
print(top_fund_house)
